# Deploying ADK Agent
## Overview

This notebook guides you on how to evaluate an ADK (Agent Development Kit) agent using Vertex AI Gen AI Evaluation for agent evaluation.

## Learning Goals

By the end of this notebook, you will understand how to:
* ADK structure and local utilities
* Deploy ADK agent to Agent Engine in Vertex AI
* Call deployed ADK agent

## Deployment Options
Your ADK agent can be deployed to a range of different environments based on your needs for production readiness or custom flexibility:

#### Agent Engine in Vertex AI
Agent Engine is a fully managed auto-scaling service on Google Cloud specifically designed for deploying, managing, and scaling AI agents built with frameworks such as ADK.

Learn more about deploying your agent to Vertex AI Agent Engine.

#### Cloud Run
Cloud Run is a managed auto-scaling compute platform on Google Cloud that enables you to run your agent as a container-based application.

Learn more about deploying your agent to Cloud Run.

#### Google Kubernetes Engine (GKE)
Google Kubernetes Engine (GKE) is a managed Kubernetes service of Google Cloud that allows you to run your agent in a containerized environment. GKE is a good option if you need more control over the deployment as well as for running Open Models.

Learn more about deploying your agent to GKE.

## Setup
This lab needs a special kernel to run, please run the following cell.
**NOTE: You can skip this step if you have already built the ADK Kernel from the previous Lab**

In [ ]:
# !echo "Kernel installation started."
# !cd ../../.. && make adk_kernel > /dev/null 2>&1
# !echo "Kernel installation completed."

When it's completed, select the **`ADK Kernel`** on the top right before going forward in the notebook.<br>
It may take ~1 minutes until the kernel is shown after the installation.

## Install Packages

In [ ]:
import asyncio
import importlib
import json
import os
import warnings

import pandas as pd
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm  # For multi-model support
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.tool_context import ToolContext
from google.genai import types  # For creating message Content/Parts
from IPython.display import HTML, Markdown, display

# Ignore all warnings
warnings.filterwarnings("ignore")

import logging

logging.basicConfig(level=logging.ERROR)

In [ ]:
LOCATION = "us-central1"
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE"  # Use Vertex AI API

In [ ]:
%%bash
echo > adk_agents/.env "GOOGLE_CLOUD_LOCATION=$GOOGLE_CLOUD_LOCATION
GOOGLE_GENAI_USE_VERTEXAI=$GOOGLE_GENAI_USE_VERTEXAI
"

In [ ]:
MODEL = "gemini-2.0-flash"

## Basic App: Weather Lookup
**NOTE: You can skip this step if you have already created "./adk_agents/agent1_weather_lookup/tools.py" file from the previous Lab**

Let's begin by building the fundamental component of our Weather Bot: a single agent capable of performing a specific task – looking up weather information. This involves creating two core pieces:

- A Tool: A Python function that equips the agent with the ability to fetch weather data.
- An Agent: The AI "brain" that understands the user's request, knows it has a weather tool, and decides when and how to use it.

### Define the Tool (get_weather)

In ADK, **Tools** are the building blocks that give agents concrete capabilities beyond just text generation. They are typically regular Python functions that perform specific actions, like calling an API, querying a database, or performing calculations.

Our first tool will provide a *mock* weather report. This allows us to focus on the agent structure without needing external API keys yet. Later, you could easily swap this mock function with one that calls a real weather service.

**Key Concept: Docstrings are Crucial\!** The agent's LLM relies heavily on the function's **docstring** to understand:

* *What* the tool does.  
* *When* to use it.  
* *What arguments* it requires (`city: str`).  
* *What information* it returns.

**Best Practice:** Write clear, descriptive, and accurate docstrings for your tools. This is essential for the LLM to use the tool correctly.

In [ ]:
%%writefile ./adk_agents/agent1_weather_lookup/tools.py
def get_weather(city: str) -> dict:
    """Retrieves the current weather report for a specified city.

    Args:
        city (str): The name of the city (e.g., "New York", "London", "Tokyo").

    Returns:
        dict: A dictionary containing the weather information.
              Includes a 'status' key ('success' or 'error').
              If 'success', includes a 'report' key with weather details.
              If 'error', includes an 'error_message' key.
    """
    print(f"--- Tool: get_weather called for city: {city} ---") # Log tool execution
    city_normalized = city.lower().replace(" ", "") # Basic normalization

    # Mock weather data
    mock_weather_db = {
        "newyork": {"status": "success", "report": "The weather in New York is sunny with a temperature of 25°C."},
        "london": {"status": "success", "report": "It's cloudy in London with a temperature of 15°C."},
        "tokyo": {"status": "success", "report": "Tokyo is experiencing light rain and a temperature of 18°C."},
    }

    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return {"status": "error", "error_message": f"Sorry, I don't have weather information for '{city}'."}

### Define the Agent (`weather_agent`)

### NOTE: You can skip this step if you have already created "./adk_agents/agent1_weather_lookup/agent.py" file from the previous Lab

Now, let's create the **Agent** itself. An `Agent` in ADK orchestrates the interaction between the user, the LLM, and the available tools.

We configure it with several key parameters:

* `name`: A unique identifier for this agent (e.g., "weather\_agent\_v1").  
* `model`: Specifies which LLM to use (e.g., `gemini-2.0-flash`).
* `description`: A concise summary of the agent's overall purpose. This becomes crucial later when other agents need to decide whether to delegate tasks to *this* agent.  
* `instruction`: Detailed guidance for the LLM on how to behave, its persona, its goals, and specifically *how and when* to utilize its assigned `tools`.  
* `tools`: A list containing the actual Python tool functions the agent is allowed to use (e.g., `[get_weather]`).

**Best Practices:** 
- Choose descriptive `name` and `description` values. These are used internally by ADK and are vital for features like automatic delegation (covered later).
- Provide clear and specific `instruction` prompts. The more detailed the instructions, the better the LLM can understand its role and how to use its tools effectively. Be explicit about error handling if needed.

In [ ]:
%%writefile ./adk_agents/agent1_weather_lookup/agent.py
from google.adk.agents import Agent
MODEL = "gemini-2.0-flash"

from .tools import get_weather

root_agent = Agent(
    name="weather_agent_v1",
    model=MODEL, # Can be a string for Gemini or a LiteLlm object
    description="Provides weather information for specific cities.",
    instruction="You are a helpful weather assistant. "
                "When the user asks for the weather in a specific city, "
                "use the 'get_weather' tool to find the information. "
                "If the tool returns an error, inform the user politely. "
                "If the tool is successful, present the weather report clearly.",
    tools=[get_weather], # Pass the function directly
)

In [ ]:
from adk_agents.agent1_weather_lookup import agent

importlib.reload(agent)  # Force reload

# Example tool usage (optional test)
print(agent.get_weather("New York"))
print(agent.get_weather("Paris"))

In [ ]:
APP_NAME = "weather_info_app"
USER_ID = "user_1"
SESSION_ID = "session_001"  # Using a fixed ID for simplicity
EXPERIMENT_NAME = "weather-agent-v1"

# Deploy to Vertex AI Agent Engine
Agent Engine is a fully managed Google Cloud service enabling developers to deploy, manage, and scale AI agents in production. 
Agent Engine handles the infrastructure to scale agents in production so you can focus on creating intelligent and impactful applications.

In [ ]:
LOCATION = "us-central1"
PROJECT = !gcloud config list --format 'value(core.project)'
PROJECT = PROJECT[0]
BUCKET_NAME = f"agent-evaluation-{PROJECT}-bucket"
BUCKET_URI = f"gs://{BUCKET_NAME}"

**Checking for the existence of BUCKET. Creating it if it doesn't exist:**

In [ ]:
!gsutil ls $BUCKET_URI || gsutil mb -l $LOCATION $BUCKET_URI

### Initialization

In [ ]:
import os

import vertexai
from absl import app, flags
from dotenv import load_dotenv
from vertexai import agent_engines
from vertexai.preview.reasoning_engines import AdkApp

vertexai.init(
    project=PROJECT,
    location=LOCATION,
    staging_bucket=BUCKET_URI,
    experiment=EXPERIMENT_NAME,
)

### Prepare your agent for Agent Engine¶
Use reasoning_engines.AdkApp() to wrap your agent to make it deployable to Agent Engine

In [ ]:
adk_app = AdkApp(agent=agent.root_agent, enable_tracing=True)

### Deploy your agent to Agent Engine
This step may take several minutes to finish.

In [ ]:

remote_app = agent_engines.create(
    adk_app,
    display_name=agent.root_agent.name,
    requirements=[
        "google-adk (>=1.5.0)",
        "google-genai (>=1.5.0,<2.0.0)",
        "pydantic (>=2.10.6,<3.0.0)",
        "absl-py (>=2.2.1,<3.0.0)",
    ],
    extra_packages=["./adk_agents/"],
)
print(f"Created remote agent: {remote_app.resource_name}")

### Try ADK agent on Agent Engine
#### Create session (remote)

In [ ]:
remote_session = remote_app.create_session(user_id=USER_ID)

#### Expected output for create_session (remote):

In [ ]:
remote_session

#### id is the session ID, and app_name is the resource ID of the deployed agent on Agent Engine.

#### List sessions (remote)

In [ ]:
remote_app.list_sessions(user_id=USER_ID)

#### Get a specific session (remote)
While using your agent locally, session ID is stored in session.id, when using your agent remotely on Agent Engine, session ID is stored in remote_session["id"].

In [ ]:
remote_app.get_session(user_id=USER_ID, session_id=remote_session["id"])

In [ ]:
#### Send queries to your agent (remote)

In [ ]:
for event in remote_app.stream_query(
    user_id=USER_ID,
    session_id=remote_session["id"],
    message="whats the weather in new york",
):
    print(event)

### Clean up
After you have finished, it is a good practice to clean up your cloud resources. 
You can delete the deployed Agent Engine instance by using the next code:

In [ ]:
remote_app.delete(force=True)

Copyright 2025 Google LLC

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.